Adapted from https://github.com/astudillo20lca/hypercognitive

**Conference Notebook Kernel: `bw25`**

In [1]:
import shutil
from pathlib import Path

import bw2data as bwd
import bw2calc as bwc
import bw2io as bwi

## Minimalistic data for an LCA

Often we do not need the full complexity of a database in our applications. In the HyperCOG project we want to implement a cyber physical system in industrial settings. So it often a good idea to keep requirements to a minimum.

This notebook creates minimalistic versions of the foreground, biosphere and impact assessment methods databases as lightweight data packages.

In [2]:
bwi.restore_project_directory("/srv/data/projects/ecoinvent38-25.tar.gz")

Restoring project backup archive - this could take a few minutes...


'ei38-teaching-25'

In [3]:
bwd.projects.set_current('ei38-teaching-25')

## biosphere database

let's imagine that in this project I am only intested in greenhouse gases and only CO2 and CH4 are emitted. Let's create a biosphere database with only those elementary flows / nodes

In [7]:
assert 'biosphere3' in bwd.databases

minimal verison of the biosphere with only the nodes I need

In [8]:
if 'mini_biosphere' in bwd.databases:

    print(f"deleting {bwd.Database('mini_biosphere').name}")
    del bwd.databases['mini_biosphere']

co2 and ch4 _nodes_ from `biosphere3`

In [9]:
co2 = next(x for x in bwd.Database("biosphere3") 
           if x['name'] == 'Carbon dioxide, fossil'
           and x['categories'] == ('air',))

ch4 = next(x for x in bwd.Database("biosphere3") 
           if x['name'] == 'Methane, fossil'
           and x['categories'] == ('air',))

In [10]:
co2_dict = {k:v for k,v in co2.as_dict().items() if k not in ['id','code']}
ch4_dict = {k:v for k,v in ch4.as_dict().items() if k not in ['id','code']}

co2_dict['code'] = 'CO2'
ch4_dict['code'] = 'CH4'
co2_dict['database'] = 'mini_biosphere'
ch4_dict['database'] = 'mini_biosphere'

bio_db = bwd.Database("mini_biosphere")
bio_db.register()

co2_new = bio_db.new_activity(**co2_dict)
co2_new.save()

ch4_new = bio_db.new_activity(**ch4_dict)
ch4_new.save()

bio_db_dpgk = bio_db.datapackage()

Successfully switch activity dataset to database `mini_biosphere`
Successfully switch activity dataset to database `mini_biosphere`


## impact assessment method

let's store a minimal version of the IPCC 2013 impact assessment method

In [11]:
# map between ef in the original biosphere database and the simplified one
ef_keys_dict = {
co2.key:co2_new.key,
ch4.key:ch4_new.key
}

ipcc2013 = bwd.Method(('IPCC 2013', 'climate change', 'GWP 100a'))
required_cfs = [(ef_keys_dict.get(key),cf) for key,cf in ipcc2013.load() if key in ef_keys_dict]
ipcc_simplified = bwd.Method(('IPCC','simple'))
ipcc_simplified.write(required_cfs)
assert ipcc_simplified.filepath_processed().is_file()

In [12]:
ipcc_simplified.register()

In [13]:
ipcc_simplified.metadata

{'abbreviation': 'ipccs.9602a5c9eb31e1a9ba91f7805bfc9e98',
 'num_cfs': 2,
 'geocollections': ['world']}

## foreground database

example of foreground database

In [15]:
a_key = ("testdb", "my_machine")

act_a_def = {
    'name': 'my_machine',
    'unit': 'kilogram',
    'exchanges': [{"input": co2_new.key, "type": "biosphere", "amount": 10},
                  {"input": a_key, "output":a_key,'type':'production','amount':1},
                  {"input": ch4_new.key, "type": "biosphere", "amount": 1},
                 ],
    }

db = bwd.Database("testdb")
db.write(
    {
    a_key : act_a_def,
    }
    )

Not able to determine geocollections for all datasets. This database is not ready for regionalization.
Title: Writing activities to SQLite3 database:
  Started: 09/27/2022 19:33:02
  Finished: 09/27/2022 19:33:02
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 0.15


calculate via normal procedure

In [16]:
lca = bwc.LCA({db.get('my_machine'):1},method=ipcc_simplified.name)

In [17]:
lca.lci()
lca.lcia()
lca.score

39.70000076293945

## get the related datapackages

we can identify the datapackages associated with our data and copy it in this folder to re-use later

In [22]:
db.backend

'sqlite'

In [23]:
db.process(csv=True)

In [24]:
bio_db.process(csv=True)

In [25]:
shutil.copyfile(src=ipcc_simplified.filepath_processed(),
                dst=Path.cwd()/ipcc_simplified.filename_processed());

In [26]:
shutil.copyfile(src=db.filepath_processed(),
                dst=Path.cwd()/db.filename_processed());

In [27]:
shutil.copyfile(
src=bio_db.filepath_processed(),
dst=Path.cwd()/bio_db.filename_processed());

Now if we look into the folder we should find some zip files with the corresponding datapackages.

In [32]:
for path in Path.cwd().glob('*zip'):
    print(path.name)

ipccs.9602a5c9eb31e1a9ba91f7805bfc9e98.zip
testdb.2a571274.zip
mini_biosphere.939a0794.zip
